<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

# Cleaning NOAA Weather Data of JFK Airport (New York)

This notebook relates to the NOAA Weather Dataset - JFK Airport (New York). The dataset contains 114,546 hourly observations of 12 local climatological variables (such as temperature and wind speed) collected at JFK airport. This dataset can be obtained for free from the IBM Developer [Data Asset Exchange](https://developer.ibm.com/exchanges/data/all/jfk-weather-data/).

In this notebook, we clean the raw dataset by:
* removing redundant columns and preserving only key numeric columns
* converting and cleaning data where required
* creating a fixed time interval between observations (this aids with later time-series analysis)
* filling missing values
* encoding certain weather features

### Table of Contents:
* [1. Read the Raw Data](#cell1)
* [2. Clean the Data](#cell2)
  * [2.1 Select data columns](#cell3)
  * [2.2 Clean up precipitation column](#cell4)
  * [2.3 Convert columns to numerical types](#cell5)
  * [2.4 Reformat and process data](#cell6)
  * [2.5 Create a fixed interval dataset](#cell7)
  * [2.6 Feature encoding](#cell8)
  * [2.7 Rename columns](#cell9)
* [3. Save the Cleaned Data](#cell10)
* [Authors](#authors)

#### Import required modules

Import and configure the required modules.

In [1]:
!pip3 install PyGithub pandas > /dev/null 2>&1

In [2]:
# Define required imports
import numpy as np
import pandas as pd
import re
import sys

# These set pandas max column and row display in the notebook
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

<a id="cell1"></a>

### 1. Read the Raw Data

We start by reading in the raw dataset, displaying the first few rows of the dataframe, and taking a look at the columns and column types present.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
raw_data = pd.read_csv('data/noaa-weather-data-jfk-airport/jfk_weather.csv',
                       parse_dates=['DATE'])
raw_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/noaa-weather-data-jfk-airport/jfk_weather.csv'

In [ ]:
raw_data.dtypes

<a id="cell2"></a>

### 2. Clean the Data

As you can see above, there are a lot of fields which are non-numerical - usually these will be fields that contain text or categorical data, e.g. `HOURLYSKYCONDITIONS`.

There are also fields - such as the main temperature field of interest `HOURLYDRYBULBTEMPF` - that we expect to be numerical, but are instead `object` type. This often indicates that there may be missing (or `null`) values, or some other unusual readings that we may have to deal with (since otherwise the field would have been fully parsed as a numerical data type).

In addition, some fields relate to hourly observations, while others relate to daily or monthly intervals. For purposes of later exploratory data analysis, we will restrict the dataset to a certain subset  of numerical fields that relate to hourly observations.

In this section, we refer to the [NOAA Local Climatological Data Documentation](https://data.noaa.gov/dataset/dataset/u-s-local-climatological-data-lcd/resource/ee7381ea-647a-434f-8cfa-81202b9b4c05) to describe the fields and meaning of various values.

<a id="cell3"></a>
#### 2.1 Select data columns

First, we select only the subset of data columns of interest and inspect the column types.

In [ ]:
# Choose what columns to import from raw data
column_subset = [
    'DATE',
    'HOURLYVISIBILITY',
    'HOURLYDRYBULBTEMPF',
    'HOURLYWETBULBTEMPF',
    'HOURLYDewPointTempF',
    'HOURLYRelativeHumidity',
    'HOURLYWindSpeed',
    'HOURLYWindDirection',
    'HOURLYStationPressure',
    'HOURLYPressureTendency',
    'HOURLYSeaLevelPressure',
    'HOURLYPrecip',
    'HOURLYAltimeterSetting'
]

# Filter dataset to relevant columns
hourly_data = raw_data[column_subset]
# Set date index
hourly_data = hourly_data.set_index(pd.DatetimeIndex(hourly_data['DATE']))
hourly_data.drop(['DATE'], axis=1, inplace=True)
hourly_data.replace(to_replace='*', value=np.nan, inplace=True)
hourly_data.head()

In [ ]:
hourly_data.dtypes

<a id="cell4"></a>
#### 2.2 Clean up precipitation column

From the dataframe preview above, we can see that the column `HOURLYPrecip` - which is the hourly measure of precipitation levels - contains both `NaN` and `T` values. `T` specifies *trace amounts of precipitation*, while `NaN` means *not a number*, and is used to denote missing values.

We can also inspect the unique values present for the field.

In [ ]:
hourly_data['HOURLYPrecip'].unique()

We can see that some values end with an `s` (indicating snow), while there is a strange value `0.020.01s` which appears to be an error of some sort. To deal with `T` values, we will set the observation to be `0`. We will also replace the erroneous value `0.020.01s` with `NaN`.

In [ ]:
# Fix imported data
hourly_data['HOURLYPrecip'].replace(to_replace='T', value='0.00', inplace=True)
hourly_data['HOURLYPrecip'].replace('0.020.01s', np.nan, inplace=True)

<a id="cell5"></a>
#### 2.3 Convert columns to numerical types

Next, we will convert string columns that refer to numerical values to numerical types. For columns such as `HOURLYPrecip`, we first also drop the non-numerical parts of the value (the `s` character).

In [ ]:
# Set of columns to convert
messy_columns = column_subset[1:]

# Convert columns to float32 datatype
for i in messy_columns:
    hourly_data[i] = hourly_data[i].apply(
        lambda x: re.sub('[^0-9,.-]', '', x)
        if type(x) == str else x).replace('', np.nan).astype(('float32'))

We can now see that all fields have numerical data type.

In [ ]:
print(hourly_data.info())
print()
hourly_data.head()

<a id="cell6"></a>
#### 2.4 Reformat and process data

Next, we will clean up some of the data columns to ensure their values fall within the parameters defined by the NOAA documentation (referred to above). 

In [ ]:
# Generate the summary statistics for each column
hourly_data.describe()

According to the documentation, the `HOURLYPressureTendency` field should be an integer value in the range `[0, 8]`. Let's check if this condition holds for this dataset.

# Check if categorical variable HOURLYPressureTendency ever has a non-integer
# entry outside the bounds of 0-8
cond =\
    len(hourly_data[~hourly_data['HOURLYPressureTendency'].isin(
        list(range(0, 9)) + [np.nan])])

print('Hourly Pressure Tendency should be between 0 and 8: {}'
      .format(cond == 0))

In [ ]:
The `HOURLYVISIBILITY` should be an integer in the range `[0, 10]`. Let's check this condition too.

In [ ]:
# Hourly Visibility should be between 0 and 10
hourly_data[(hourly_data['HOURLYVISIBILITY'] < 0) | (hourly_data['HOURLYVISIBILITY'] > 10)]

We find that a couple of observations fall outside the range. These must be spurious data observations and we handle them by replacing them with `NaN`.

In [ ]:
# Replace any hourly visibility figure outside these bounds with nan
hourly_data.loc[hourly_data['HOURLYVISIBILITY'] > 10, 'HOURLYVISIBILITY'] = np.nan

# Hourly Visibility should be between 0 and 10
cond = len(hourly_data[(hourly_data['HOURLYVISIBILITY'] < 0) | (hourly_data['HOURLYVISIBILITY'] > 10)])

print('Hourly Visibility should be between 0 and 10: {}'.format(cond == 0))

Finally, we check if there are any duplicates with respect to our `DATE` index and check furthermore that our dates are in the correct order (that is, strictly increasing).

In [ ]:
cond = len(hourly_data[hourly_data.index.duplicated()].sort_index())
print('Date index contains no duplicate entries: {}'.format(cond == 0))

In [ ]:
# Make sure time index is sorted and increasing
print('Date index is strictly increasing: {}'
      .format(hourly_data.index.is_monotonic_increasing))

<a id="cell7"></a>
#### 2.5 Create a fixed interval dataset

Most time-series analysis requires (or certainly works much better with) data that has fixed measurement intervals. As you may have noticed from the various data samples above, the measurement intervals for this dataset are not exactly hourly. So, we will use `Pandas`' resampling functionality to create a dataset that has exact hourly measurement intervals.

In [ ]:
# Resample (downsample) to hourly rows (we're shifting everything up by 9 minutes!)
hourly_data = hourly_data.resample('60min').last().shift(periods=1)  # noqa Note: use resample('60min', base=51) to resample on the 51st of every hour

We will now also replace missing values. For numerical values, we will linearly interpolate between the previous and next valid obvservations. For the categorical `HOURLYPressureTendency` field, we will replace missing values with the last valid observation.

In [ ]:
hourly_data['HOURLYPressureTendency'] =\
    hourly_data['HOURLYPressureTendency'].fillna(method='ffill')  # fill with last valid observation
hourly_data = hourly_data.interpolate(method='linear')  # interpolate missing values
hourly_data.drop(hourly_data.index[0], inplace=True)  # drop first row

In [ ]:
print(hourly_data.info())
print()
hourly_data.head()

<a id="cell8"></a>
#### 2.6 Feature encoding

The final pre-processing step we will perform will be to handle two of our columns in a special way in order to correctly encode these features. They are:

1. `HOURLYWindDirection` - wind direction
2. `HOURLYPressureTendency` - an indicator of pressure changes

For `HOURLYWindDirection`, we encode the raw feature value as two new values, which measure the cyclical nature of wind direction - that is, we are encoding the compass-point nature of wind direction measurements.

In [ ]:
# Transform HOURLYWindDirection into a cyclical variable using sin and cos transforms
hourly_data['HOURLYWindDirectionSin'] = np.sin(hourly_data['HOURLYWindDirection'] * (2. * np.pi / 360))
hourly_data['HOURLYWindDirectionCos'] = np.cos(hourly_data['HOURLYWindDirection'] * (2. * np.pi / 360))
hourly_data.drop(['HOURLYWindDirection'], axis=1, inplace=True)

For `HOURLYPressureTendency`, the feature value is in fact a `categorical` feature with three levels:
* `0-3` indicates an increase in pressure over the previous 3 hours
* `4` indicates no change during the previous 3 hours
* `5-8` indicates a decrease over the previous 3 hours

Hence, we encode this feature into 3 dummy values representing these 3 potential states.

In [ ]:
# Transform HOURLYPressureTendency into 3 dummy variables based on NOAA documentation
hourly_data['HOURLYPressureTendencyIncr'] =\
    [1.0 if x in [0, 1, 2, 3]
        else 0.0 for x in hourly_data['HOURLYPressureTendency']]  # noqa 0 through 3 indicates an increase in pressure over previous 3 hours
hourly_data['HOURLYPressureTendencyDecr'] =\
    [1.0 if x in [5, 6, 7, 8]
     else 0.0 for x in hourly_data['HOURLYPressureTendency']]  # noqa 5 through 8 indicates a decrease over previous 3 hours
hourly_data['HOURLYPressureTendencyConst'] =\
    [1.0 if x == 4
     else 0.0 for x in hourly_data['HOURLYPressureTendency']]  # noqa 4 indicates no change during previous 3 hours
hourly_data.drop(['HOURLYPressureTendency'], axis=1, inplace=True)
hourly_data['HOURLYPressureTendencyIncr'] =\
    hourly_data['HOURLYPressureTendencyIncr'].astype(('float32'))
hourly_data['HOURLYPressureTendencyDecr'] =\
    hourly_data['HOURLYPressureTendencyDecr'].astype(('float32'))
hourly_data['HOURLYPressureTendencyConst'] =\
    hourly_data['HOURLYPressureTendencyConst'].astype(('float32'))

<a id="cell9"></a>
#### 2.7 Rename columns

Before saving the dataset, we will rename the columns for readability.

In [ ]:
hourly_data.columns

In [ ]:
# define the new column names
columns_new_name = [
    'visibility',
    'dry_bulb_temp_f',
    'wet_bulb_temp_f',
    'dew_point_temp_f',
    'relative_humidity',
    'wind_speed',
    'station_pressure',
    'sea_level_pressure',
    'precip',
    'altimeter_setting',
    'wind_direction_sin',
    'wind_direction_cos',
    'pressure_tendency_incr',
    'pressure_tendency_decr',
    'pressure_tendency_const'
]

columns_name_map =\
    {c: columns_new_name[i] for i, c in enumerate(hourly_data.columns)}

hourly_data_renamed = hourly_data.rename(columns=columns_name_map)

In [ ]:
print(hourly_data_renamed.info())
print()
hourly_data_renamed.head()

In [ ]:
# Explore some general information about the dataset
print('# of megabytes held by dataframe: {}'.format(
      str(round(sys.getsizeof(hourly_data_renamed) / 1000000, 2))))
print('# of features: {}'.format(str(hourly_data_renamed.shape[1])))
print('# of observations: {}'.format(str(hourly_data_renamed.shape[0])))
print('Start date: {}'.format(str(hourly_data_renamed.index[0])))
print('End date: {}'.format(str(hourly_data_renamed.index[-1])))
print('# of days: {}'.format(
      str((hourly_data_renamed.index[-1] - hourly_data_renamed.index[0]).days)))
print('# of months: {}'.format(
      str(round((hourly_data_renamed.index[-1] - hourly_data_renamed.index[0]).days / 30, 2))))
print('# of years: {}'.format(
      str(round((hourly_data_renamed.index[-1] - hourly_data_renamed.index[0]).days / 365, 2))))

<a id="cell10"></a>

### 3. Save the Cleaned Data

Finally, we save the cleaned dataset as a Project asset for later re-use. You should see an output like the one below if successful:

```
{'file_name': 'jfk_weather_cleaned.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'jfkweatherdata-donotdelete-pr-...',
 'asset_id': '...'}
```

**Note**: In order for this step to work, your project token (see the first cell of this notebook) must have `Editor` role. By default this will overwrite any existing file.

In [ ]:
hourly_data_renamed.to_csv(
    "data/noaa-weather-data-jfk-airport/jfk_weather_cleaned.csv",  
    float_format='%g')

#### Next steps

- Close this notebook.
- Open the `Part 2 - Data Analysis` notebook to explore the cleaned dataset.

<a id="authors"></a> 
### Authors

This notebook was created by the [Center for Open-Source Data & AI Technologies](http://codait.org).

Copyright © 2019 IBM. This notebook and its source code are released under the terms of the MIT License.